In [35]:
from hatchet import *
import networkx as nx
import pandas
from hatchet import *
import os
import matplotlib.pyplot as plt
import seaborn.apionly as sns
import igraph
import platform
import json
import re
from ast import literal_eval as make_tuple

In [36]:
from state import State

In [37]:
dirname = "/Users/jarus/ucd/Research/Visualisation/projects/CallFlow/.callflow"

In [38]:
def replace_str_with_Node(df, graph):
        mapper = {}
        def dfs_recurse(root):
            for node in root.children: 
                mapper[node.callpath[-1]] = Node(node.nid, node.callpath, None)
                dfs_recurse(node)
        for root in graph.roots:
            mapper[root.callpath[-1]] = Node(root.nid, root.callpath, None)
            dfs_recurse(root)
        df['node'] = df['node'].apply(lambda node: mapper[node] if node in mapper else '')
        return df

In [39]:
def read_gf(name):
    state = State()
    df_filepath = dirname + '/' + name +  '/filter_df.csv'
    entire_df_filepath = dirname + '/' + name + '/entire_df.csv'
    graph_filepath = dirname + '/' + name + '/filter_graph.json'
    entire_graph_filepath = dirname + '/' + name + '/entire_graph.json'   

    with open(graph_filepath, 'r') as graphFile:
        data = json.load(graphFile)

    state.gf = GraphFrame()
    state.gf.from_literal(data)

    with open(entire_graph_filepath, 'r') as entire_graphFile:
        entire_data = json.load(entire_graphFile)
            
    state.entire_gf = GraphFrame()
    state.entire_gf.from_literal(entire_data)

    state.df = pd.read_csv(df_filepath)
    state.entire_df = pd.read_csv(entire_df_filepath)

    state.graph = state.gf.graph
    state.entire_graph = state.entire_gf.graph

    state.map = state.node_hash_mapper()

    # Print the module group by information. 
    # print(state.df.groupby(['module']).agg(['mean','count']))

    # replace df['node'] from str to the Node object.
    state.df = replace_str_with_Node(state.df, state.graph)
    state.entire_df = replace_str_with_Node(state.entire_df, state.entire_graph)
    
    group_graph_filepath = dirname + '/' + name + '/group_df.csv'
    state.gdf = pd.read_csv(group_graph_filepath)
    return state

In [40]:
datasets = ["osu_bw"]
states = {}
for idx, dataset_name in enumerate(datasets):
    state = read_gf(dataset_name)
    states[dataset_name] = state

In [41]:
df = states['osu_bw'].gdf
node_df = df.loc[df['nid'] == node_id]
print(node_df)

    Unnamed: 0                    node_index  rank_index  time (inc)  \
0            0                <program root>           0   2293720.0   
1            1                <program root>           1   2297724.0   
2            2                          main           0   2293720.0   
3            3                          main           1   2297724.0   
4            4             Loop@osu_bw.c:112           0   1293311.0   
5            5              122:PMPI_Waitall           0   1293311.0   
6            6         309:MPIR_Waitall_impl           0   1293311.0   
7            7  145:MPIDI_CH3_Progress_start           0    466412.0   
8            8       189:<unknown procedure>           0    466412.0   
9            9           <unknown procedure>           0    466412.0   
10          10           <unknown procedure>           0    466412.0   
11          11           <unknown procedure>           0    466412.0   
12          12           <unknown procedure>           0    4664